# Introduction to some forms of parallelism

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import time
import threading as th

## Fibonacci sequence

$$
F_n = \left\{
    \begin{array}{rl}
        0, & n = 0\\
        1, & n = 1\\
        F_{n - 1} + F_{n - 2}, & n > 1
    \end{array}
\right.
$$

In [4]:
def fibonacci(n: int):
    if n == 0 or n == 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [14]:
from queue import Queue

class FibonacciParallel:
    def __init__(self, n_threads: int):
        self.n_threads = n_threads

    def run(self, input_list: list|tuple):
        self.input_list = input_list
        self.fibo_dict = {}
        self.shared_queue = Queue()
        self.condition = th.Condition()
        self.threads = []
        for _ in range(self.n_threads):
            self.threads.append(th.Thread(daemon=True, target=self.__fibonacci_task))
            self.threads[-1].start()
        th.Thread(name='queue_task_thread', daemon=True, target=self.__queue_task).start()
        for thread in self.threads: thread.join()

    def __fibonacci_task(self):
        print('starting queue_task...')
        with self.condition:
            while self.shared_queue.empty():
                self.condition.wait()
                print(f'[{th.current_thread().name}] - waiting for elements in queue...')
            else:
                value = self.shared_queue.get()
                self.fibo_dict[value] = fibonacci(value)
            self.shared_queue.task_done()
            print(f'[{th.current_thread().name}] - fibonacci of key {value} with result {self.fibo_dict[value]}')
    
    def __queue_task(self):
        print('Starting queue_task...')
        with self.condition:
            for item in self.input_list:
                self.shared_queue.put(item)
                print('Notifying fibonacci_task threads that queue is ready to consume...')
                self.condition.notify_all()

In [26]:
x = [30, 32, 36, 38, 40]
fibo = FibonacciParallel(4)

In [27]:
%%time
fibo.run(x)
fibo.fibo_dict

starting queue_task...
starting queue_task...
starting queue_task...
starting queue_task...
Starting queue_task...
Notifying fibonacci_task threads that queue is ready to consume...
Notifying fibonacci_task threads that queue is ready to consume...
Notifying fibonacci_task threads that queue is ready to consume...
Notifying fibonacci_task threads that queue is ready to consume...
Notifying fibonacci_task threads that queue is ready to consume...
[Thread-40 (__fibonacci_task)] - waiting for elements in queue...
[Thread-40 (__fibonacci_task)] - fibonacci of key 30 with result 832040
[Thread-42 (__fibonacci_task)] - waiting for elements in queue...
[Thread-42 (__fibonacci_task)] - fibonacci of key 32 with result 2178309
[Thread-41 (__fibonacci_task)] - waiting for elements in queue...
[Thread-41 (__fibonacci_task)] - fibonacci of key 36 with result 14930352
[Thread-43 (__fibonacci_task)] - waiting for elements in queue...
[Thread-43 (__fibonacci_task)] - fibonacci of key 38 with result 39

{30: 832040, 32: 2178309, 36: 14930352, 38: 39088169}

In [28]:
%%time
{val: fibonacci(val) for val in x}

CPU times: user 23.9 s, sys: 0 ns, total: 23.9 s
Wall time: 23.9 s


{30: 832040, 32: 2178309, 36: 14930352, 38: 39088169, 40: 102334155}